# On the importance of Dualization

In [1]:
using DynamicPolynomials
using SumOfSquares

Sum-of-Squares programs are usually solved by SemiDefinite Programming solvers (SDPs).
These programs can be represented into two different formats:
Either the *standard conic form* or *kernel form*:
$$
\begin{aligned}
  \min\limits_{Q \in \mathbb{S}_n} & \la C, Q \ra\\
  \text{subject to:} & \la A_i, Q \ra = b_i, \quad i=1,2,\ldots,m\\
                      & Q \succeq 0,
\end{aligned}
$$
Either the *geometric conic form* or *image form*:
$$
\begin{aligned}
  \max\limits_{y \in \mathbb{R}^m} & \la b, y \ra\\
  \text{subject to:} & C \succeq \sum_{i=1}^m A_i y_i\\
                     & y\ \mathsf{free},
$$

In this tutorial, we investigate in which of these two forms a Sum-of-Squares
constraint should be written into.
Consider the simple example of trying to see whether the following univariate
polynomial is Sum-of-Squares:

In [2]:
import SCS
@polyvar x
p = (x + 1)^2 * (x + 2)^2
model_scs = Model(SCS.Optimizer)
con_ref = @constraint(model_scs, p in SOSCone())
optimize!(model_scs)

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 6, constraints m: 11
cones: 	  z: primal zero / dual free vars: 5
	  s: psd vars: 6, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 12, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.30e+01  1.87e+00  2.95e+01  1.47e+01  1.00e-01  1.89e-04 
    50| 6.01e-07  2.84e-07  1.95e-06 -9.76e-07  1.00e-01  2.96e-04 
---------------------------------

As we can see in the log, SCS reports `6` variables and `11` constraints.
We can also choose to dualize the problem before it is
passed to SCS as follows:

In [3]:
using Dualization
model_dual_scs = Model(dual_optimizer(SCS.Optimizer))
@objective(model_dual_scs, Max, 0.0)
con_ref = @constraint(model_dual_scs, p in SOSCone())
optimize!(model_dual_scs)

------------------------------------------------------------------
	       SCS v3.2.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 5, constraints m: 6
cones: 	  s: psd vars: 6, ssize: 1
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 6, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.97e+02  1.05e+01  4.39e+03 -2.20e+03  1.00e-01  7.37e-05 
    50| 6.29e-06  1.81e-06  2.22e-05 -1.11e-05  1.00e-01  1.71e-04 
------------------------------------------------------------------
status

This time, SCS reports `5` variables and `6` constraints.

In [4]:
print_active_bridges(model_scs)

 * Supported objective: MOI.ScalarAffineFunction{Float64}
 * Unsupported constraint: MOI.VectorAffineFunction{Float64}-in-SumOfSquares.SOSPolynomialSet{FullSpace, DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, DynamicPolynomials.MonomialVector{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}, SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, SumOfSquares.Certificate.NewtonFilter{SumOfSquares.Certificate.NewtonDegreeBounds{Tuple{}}}}}
 |  bridged by:
 |   SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Float64, MOI.VectorAffineFunction{Float64}, FullSpace, Union{MOI.ConstraintIndex{MOI.VectorOfVariables, MOI.Nonnegatives}, MOI.ConstraintIndex{MOI.VectorOfVariables, MOI.PositiveSemidefiniteConeTriangle}, MOI.ConstraintIndex{MOI.VectorOfVariables, SumOfSquares.EmptyCone}, MOI.Constrain

This time, we had

## In more details

Consider a polynomial
$$
p(x) = \sum_{\alpha} p_\alpha x^\alpha,
$$
a vector of monomials `b(x)` and the set
$$
\mathcal{A}_\alpha = \{\,(\beta, \gamma) \in b(x)^2 \mid x^\beta x^\gamma = x^\alpha\,\}
$$
The constraint that there exists a PSD matrix `Q` such that `p(x) = b(x)' * Q * b(x)`
can be written in standard conic form as follows:
$$
\begin{aligned}
  \langle \sum_{(\beta, \gamma) \in \mathcal{A}_\alpha} e_\beta e_\gamma^\top, Q \rangle & = p_\alpha, \quad\forall \alpha\\
  Q & \succeq 0
\end{aligned}
$$
Given an arbitrary choice of elements in each set $\mathcal{A}_\alpha$:
$(\beta_\alpha, \gamma_\alpha) \in \mathcal{A}_\alpha$.
It can also equivalently be written in the geometric conic form as follows:
$$
\begin{aligned}
  p_\alpha e_{\beta_\alpha} e_{\gamma_\alpha}^\top +
  \sum_{(\beta, \gamma) \in \mathcal{A}_\alpha \setminus (\beta_\alpha, \gamma_\alpha)}
  y_{\beta,\gamma} (e_\beta e_\gamma - e_{\beta_\alpha} e_{\gamma_\alpha}^\top)^\top
  & \succeq 0\\
  y_{\beta,\gamma} & \text{ free}
\end{aligned}
$$

## Which one do I choose ?

Let's pick two extreme examples and then try to extrapolate from these.

### Univariate case

Suppose `p` is a univariate polynomial of degree $2d$.
Then `n` will be equal to `d(d + 1)/2` for both the standard and geometric conic forms.
On the other hand, `m` will be equal to `2d + 1` for the standard conic form and
`d(d + 1) / 2 - (2d + 1)` for the geometric form case.
So `m` grows **linearly** for the kernel form but **quadratically** for the image form!

### Quadratic case

Suppose `p` is a quadratic form of `d` variables.
Then `n` will be equal to `d` for both the standard and geometric conic forms.
On the other hand, `m` will be equal to `d(d + 1)/2` for the standard conic form and
`0` for the geometric form case.
So `m` grows **quadratically** for the kernel form but is zero for the image form!

### In general

In general, if $s_d$ is the dimension of the space of polynomials of degree `d` then
$m = s_{2d}$ for the kernel form and $m = s_{d}(s_{d} + 1)/2 - s_{2d}$ for the image form.
As a rule of thumb, the kernel form will have a smaller `m` if `p` has a low number of variables
and low degree and vice versa.
Of course, you can always try with and without Dualization and see which one works best.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*